In [13]:
# ==============================================================================
# CÉLULA 1: INSTALAÇÃO DE DEPENDÊNCIAS E AUTENTICAÇÃO
# ==============================================================================

print("Instalando dependências...")
!pip install earthengine-api --quiet
!pip install geopandas --quiet
!pip install rasterio --quiet
!pip install shap --quiet
!pip install geemap --quiet
print("Dependências instaladas.")

# ------------------------------------------------------------------------------
# Autenticação e Inicialização de Serviços Google
# ------------------------------------------------------------------------------
from google.colab import drive
import ee

# A autenticação com o Earth Engine pedirá um token de autorização.
try:
    ee.Initialize(project="the-byway-476116-n7")
    print("API do Google Earth Engine já inicializada.")
except Exception as e:
    print("Autenticando na API do Google Earth Engine...")
    ee.Authenticate()
    ee.Initialize(project="the-byway-476116-n7")
    print("API do Google Earth Engine inicializada.")

# Monta o Google Drive para que possamos salvar arquivos (datasets, modelos)
print("Montando Google Drive...")
drive.mount('/content/drive')
print("Google Drive montado em /content/drive")

Instalando dependências...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.0 MB/s eta 0:00:00
Dependências instaladas.
API do Google Earth Engine já inicializada.
Montando Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado em /content/drive


In [4]:
# ==============================================================================
# CÉLULA 2: IMPORTAÇÃO DAS BIBLIOTECAS PRINCIPAIS
# ==============================================================================
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import torch

print("Bibliotecas principais importadas. Ambiente pronto para uso!")

Bibliotecas principais importadas. Ambiente pronto para uso!


In [55]:
# ==============================================================================
# PASSO 1: DEFINIÇÃO DOS PARÂMETROS GLOBAIS
# ==============================================================================

# 1. Período de Tempo para Análise(10 anos)
START_DATE = '2014-01-01'
END_DATE = '2024-01-01'

# 2. Área de Interesse(AOI)
# Coleção de biomas do Brasil carregada e filtrada para obter apenas o polígono do Cerrado.
# Este polígono 'cerrado_aoi' será usado para "recortar" todos os outros dados.
biomas = ee.FeatureCollection('projects/mapbiomas-workspace/AUXILIAR/ESTATISTICAS/COLECAO8/VERSAO-1/refined_biome')
cerrado_aoi = biomas.filter(ee.Filter.eq('NAME_PT_BR', 'Cerrado')).geometry()

# 3. Resolução Espacial(30m)
TARGET_RESOLUTION = 30

print("Parâmetros definidos:")
print(f"Período de análise: {START_DATE} a {END_DATE}")
print("Área de Interesse: Bioma Cerrado")

Parâmetros definidos:
Período de análise: 2014-01-01 a 2024-01-01
Área de Interesse: Bioma Cerrado


In [80]:
# ==============================================================================
# PASSO 2: COLETA DA VARIÁVEL ALVO (ÁREA QUEIMADA)
# ==============================================================================

# 1. Carregamos o asset global como uma única ee.Image, SEM NENHUM CORTE.
burned_area_image_global = ee.Image('projects/mapbiomas-public/assets/brazil/fire/collection4/mapbiomas_fire_collection4_monthly_burned_v1')

# --- TESTE DE VERIFICAÇÃO SEGURO ---
# Agora fazemos as perguntas ao objeto global e simples.
print("Iniciando verificação segura no objeto global...")

# Nomes de todas as bandas do objeto global.
band_names_server = burned_area_image_global.bandNames()

# Pedimos ao servidor para nos enviar o NÚMERO TOTAL de bandas.
num_bands = band_names_server.size().getInfo()
print(f"O objeto global é uma imagem única com {num_bands} bandas.")

# Pedimos ao servidor para "fatiar" a lista e nos enviar APENAS OS 5 PRIMEIROS NOMES.
first_5_bands = band_names_server.slice(0, 5).getInfo()
print("Exemplo dos primeiros 5 nomes de banda:", first_5_bands)
print("Verificação concluída com sucesso.")

# ------------------------------------------------------------------------------

# 2. Objeto cortado para uso futuro
burned_area_cerrado = burned_area_image_global.clip(cerrado_aoi)

print("\nObjeto 'burned_area_cerrado' (recortado para o Cerrado) criado para uso futuro.")

Iniciando verificação segura no objeto global...
O objeto global é uma imagem única com 40 bandas.
Exemplo dos primeiros 5 nomes de banda: ['burned_monthly_1985', 'burned_monthly_1986', 'burned_monthly_1987', 'burned_monthly_1988', 'burned_monthly_1989']
Verificação concluída com sucesso.

Objeto 'burned_area_cerrado' (recortado para o Cerrado) criado para uso futuro.


In [78]:
# ==============================================================================
# PASSO 3: COLETA DAS VARIÁVEIS PREDITIVAS (FEATURES)
# ==============================================================================

# --- 3.1 DADOS METEOROLÓGICOS (ERA5-Land) ---
# Contém dados de temperatura, umidade, vento, etc.
era5_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
   .filterDate(START_DATE, END_DATE) \
   .filterBounds(cerrado_aoi)

print(f"Encontradas {era5_collection.size().getInfo()} imagens horárias de meteorologia.")

# --- 3.2 DADOS DE VEGETAÇÃO (MODIS & Landsat) ---
# NDVI/EVI (saúde da vegetação) e LST (temperatura da superfície) do MODIS.
ndvi_collection = ee.ImageCollection('MODIS/061/MOD13A2') \
   .filterDate(START_DATE, END_DATE) \
   .filterBounds(cerrado_aoi) \
   .select('NDVI')

lst_collection = ee.ImageCollection('MODIS/061/MOD11A1') \
   .filterDate(START_DATE, END_DATE) \
   .filterBounds(cerrado_aoi) \
   .select('LST_Day_1km')

print(f"Encontradas {ndvi_collection.size().getInfo()} imagens de NDVI (16 dias).")
print(f"Encontradas {lst_collection.size().getInfo()} imagens de LST (diárias).")

# --- 3.3 DADOS TOPOGRÁFICOS (SRTM) ---
# Elevação é uma imagem única (estática), não uma coleção.
elevation = ee.Image('USGS/SRTMGL1_003').clip(cerrado_aoi)
# Cálculo do declive (slope) diretamente a partir da elevação.
slope = ee.Terrain.slope(elevation)

print("Dados de elevação e declive carregados.")

# --- 3.4 DADOS ANTROPOGÊNICOS (MapBiomas) ---
# Uso e cobertura do solo.
land_cover = ee.Image('projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_coverage_v2').clip(cerrado_aoi)

print("Dados de uso e cobertura do solo carregados.")

Encontradas 87648 imagens horárias de meteorologia.
Encontradas 230 imagens de NDVI (16 dias).
Encontradas 3631 imagens de LST (diárias).
Dados de elevação e declive carregados.
Dados de uso e cobertura do solo carregados.
